# Formation Energy Lanscape of Cubic Perovskites

The formation energy ($\Delta E_{f}^{Pm\bar{3}m}$) of the perovskite structure is then calculated with respect to the energies of its constituent atoms as,
\begin{equation}
    \Delta E_f^{Pm\bar{3}m}\left(\mbox{A}\mbox{B}\mbox{X}_{3}\right)=\frac{1}{N}\left[ E_{ABX_{3}}-E_{A}-E_{B}-3E_{X} \right],
\end{equation}
in which $E$ represents the total  energy obtained from DFT, and is normalised by the total number of atoms $N$ in a simulation cell.

Our primary interest here is the relative stabilities of the cubic perovskite phase, which is governed by its convex-hull stabilities ($\Delta H_{c}$). However, this would require the knowledge of $\Delta E_{f}$ for the most stable phase for every compound with ABX$_{3}$ stoichiometry. This information is not available for all materials to be screened here, which is particularly true for hypothetical materials. Although performing structure predictions across different crystallographic space groups can help providing a more accurate evaluation on the relative phase stabilities,  it would be computationally too demanding to follow this route for every new compound investigated in this work. As such, following the approach of *J. Phys. Chem. A* **123**, 7323 (2019), we proximate $\Delta H_{c}$ as:
\begin{equation}
    \Delta H_c=\Delta E_f^{Pm\bar{3}m}-\min\left\{\Delta E_{f,i}^{\mbox{rand}}, i\in[1,10] \right\},
\end{equation}

Here, starting from the optimized structure of a cubic perovskite, 10 random crystal structures are generated by arbitrarily distorting its lattice parameters and atomic positions. The randomly generated structure is  then fully optimized without any constraints. The lowest energy within these 10 random structures gives the second term in the above equation. As such, $\Delta H_c$ effectively measures the convex hull stability of cubic perovskite with respect to a hypothetical $P1$ structure with the same composition. 

This notebook enable you to inspect the trend of $\Delta H_c$ for all perovskites stored in this database. The values of $\Delta H_c$ are plotted against the Goldschmidt tolerance factors for the cubic perovskites.

In [37]:
from tolerance_factor import tolerance_factor
from chemical_space import *
from ase.db import connect

Specify the energy landscape for which subset of compounds you would like to inspect, as categorized by the identity of the anion X in the ABX$_{3}$ perovskite.

In [38]:
X='O'

#get the corresponding A, B cations that go with the anion
if X in halide_C:
    A = halide_A
    B = halide_B
elif X in chalco_C:
    A = chalco_A
    B = chalco_B

#connect to the corresponding database toto retrieve the data
db=connect('./perovskites_'+str(X)+'.db')

In [76]:
tolerance_factors=[]
energy_differences=[] #as the formation energies of Pm3m perovskite to the most stable random structure
label=[]

for a in A:
    for b in B:
        row = None
        system_name = a + b + X
        uid = system_name + '_Pm3m'
        pm3m_formation_e = None
        
        tolerance_f = tolerance_factor(a, b, X)
        
        #Get the database entry corresponding to this cubic perovskite
        try:
            row = db.get(selection=[('uid', '=', uid)])
        except:
            continue
            
        if row is not None:
            #formation energy for the cubic perovskite
            try:
                pm3m_formation_e = row.key_value_pairs['formation_energy']
            except KeyError:
                continue
                
            if pm3m_formation_e is not None:
                #Get the formation energies for the most stable structure that is optimized from a random starting 
                #point. In the actual computational screening procedure, 10 structures were screened for each compound.
                #However, due to the storage limit of the Github repository, only the information on the 
                #lowest energy structure is stored. This bit of the code is trying to digging that information out
                random_energy = None
                counter = 0
                
                while (random_energy is None) or (counter<10):
                    uid_r = uid + '_rand_str_' + str(counter)
                    try:
                        row = db.get(selection=[('uid', '=', uid_r)])
                    except:
                        pass
                    
                    if row is not None:
                        random_energy = row.key_value_pairs['formation_energy']
                    counter+=1

                if random_energy is not None:
                    energy_differences.append(pm3m_formation_e - random_energy)
                    tolerance_factors.append(tolerance_f)
                    label.append(system_name+'3')
print("Total number of available data: "+str(len(energy_differences)))

Total number of available data: 507


In [77]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import Div
from bokeh.models.tools import HoverTool
from bokeh.models import ColumnDataSource    

output_notebook()
p = figure()
p.line([max(tolerance_factors),min(tolerance_factors)],[0,0],color='blue')

data = {'x':tolerance_factors,
        'y':energy_differences,
        'label':label}
source = ColumnDataSource(data=data)

p.circle(x='x', y='y', source=source,size=10,color='#CB0000',alpha=0.6)

hover = HoverTool()
hover.tooltips=[
    ('t', '@x'),
    ('Delta Hc', '@y'),
    ('Compound', '@label')
]
p.add_tools(hover)

p.xaxis.axis_label='Tolerance Factor'
p.yaxis.axis_label="Convex Hull Energies (eV/atom)"
p.xaxis.axis_label_text_font_size = "20pt"
p.yaxis.axis_label_text_font_size = "16pt"
p.xaxis.major_label_text_font_size = "15pt"
p.yaxis.major_label_text_font_size = "15pt"
show(p)

Loading BokehJS ...

The following provides a matrix plot to show the formation energy lanscapes of pervoskites with a specific type of anion.

In [ ]:
import numpy as np
from bokeh.transform import linear_cmap
from bokeh.palettes import Spectral6


data_grid = np.zeros((len(B),len(A)))
label=[]
alphas=np.zeros((len(B),len(A)))
colors=np.zeros((len(B),len(A)))
x_name=[]
y_name=[]
for b_count, b in enumerate(B):
    for a_count, a in enumerate(A):
        x_name.append(a)
        y_name.append(b)
        data_grid[b_count][a_count] = 100
        system_name = a + b + X
        uid = system_name + '_Pm3m'
        label.append(system_name+'3')
        row = None
        pm3m_formation_e = None
        
        #Get the database entry corresponding to this cubic perovskite
        try:
            row = db.get(selection=[('uid', '=', uid)])
        except:
            continue
            
        if row is not None:
            #formation energy for the cubic perovskite
            try:
                pm3m_formation_e = row.key_value_pairs['formation_energy']
            except KeyError:
                continue
                
            if pm3m_formation_e is not None:
                #Get the formation energies for the most stable structure that is optimized from a random starting 
                #point. In the actual computational screening procedure, 10 structures were screened for each compound.
                #However, due to the storage limit of the Github repository, only the information on the 
                #lowest energy structure is stored. This bit of the code is trying to digging that information out
                random_energy = None
                counter = 0
                
                while (random_energy is None) or (counter<10):
                    uid_r = uid + '_rand_str_' + str(counter)
                    try:
                        row = db.get(selection=[('uid', '=', uid_r)])
                    except:
                        pass
                    
                    if row is not None:
                        random_energy = row.key_value_pairs['formation_energy']
                    counter+=1

                if random_energy is not None:
                    data_grid[b_count,a_count] = pm3m_formation_e - random_energy
                    

                    
min_energy = min([i for i in data_grid.flatten() if i!=100])
max_energy = max([i for i in data_grid.flatten() if i!=100])

mapper = linear_cmap(field_name='energy', palette=Spectral6 ,low=min_energy ,high=min_energy)

for b_count, b in enumerate(B):
    for a_count, a in enumerate(A):
        this_en = data_grid[b_count,a_count]
        if this_en != 100:
            alphas[b_count,a_count] = (this_en - min_energy)/(max_energy-min_energy)
            colors[b_count,a_count] = (this_en - min_energy)/(max_energy-min_energy)
        else:
            alphas[b_count,a_count] = 0.1
            colors[b_count,a_count] = 0
        

data=dict(
    xname=x_name,
    yname=y_name,
    colors=['r' for 

In [113]:
import numpy as np
from bokeh.transform import linear_cmap
from bokeh.palettes import Spectral6


data_grid = np.zeros((len(B),len(A)))
label=[]
alphas=np.zeros((len(B),len(A)))
colors=np.zeros((len(B),len(A)))
x_name=[]
y_name=[]
for b_count, b in enumerate(B):
    for a_count, a in enumerate(A):
        x_name.append(a)
        y_name.append(b)
        data_grid[b_count][a_count] = 100
        system_name = a + b + X
        uid = system_name + '_Pm3m'
        label.append(system_name+'3')
        row = None
        pm3m_formation_e = None
        
        #Get the database entry corresponding to this cubic perovskite
        try:
            row = db.get(selection=[('uid', '=', uid)])
        except:
            continue
            
        if row is not None:
            #formation energy for the cubic perovskite
            try:
                pm3m_formation_e = row.key_value_pairs['formation_energy']
            except KeyError:
                continue
                
            if pm3m_formation_e is not None:
                #Get the formation energies for the most stable structure that is optimized from a random starting 
                #point. In the actual computational screening procedure, 10 structures were screened for each compound.
                #However, due to the storage limit of the Github repository, only the information on the 
                #lowest energy structure is stored. This bit of the code is trying to digging that information out
                random_energy = None
                counter = 0
                
                while (random_energy is None) or (counter<10):
                    uid_r = uid + '_rand_str_' + str(counter)
                    try:
                        row = db.get(selection=[('uid', '=', uid_r)])
                    except:
                        pass
                    
                    if row is not None:
                        random_energy = row.key_value_pairs['formation_energy']
                    counter+=1

                if random_energy is not None:
                    data_grid[b_count,a_count] = pm3m_formation_e - random_energy
                    

                    
min_energy = min([i for i in data_grid.flatten() if i!=100])
max_energy = max([i for i in data_grid.flatten() if i!=100])

mapper = linear_cmap(field_name='energy', palette=Spectral6 ,low=min_energy ,high=min_energy)

for b_count, b in enumerate(B):
    for a_count, a in enumerate(A):
        this_en = data_grid[b_count,a_count]
        if this_en != 100:
            alphas[b_count,a_count] = (this_en - min_energy)/(max_energy-min_energy)
            colors[b_count,a_count] = (this_en - min_energy)/(max_energy-min_energy)
        else:
            alphas[b_count,a_count] = 0
            colors[b_count,a_count] = 0
            data_grid[b_count,a_count] = np.NaN

data=dict(
    xname=x_name,
    yname=y_name,
    #colors=colors.flatten(),
    alphas=alphas.flatten(),
    count=data_grid.flatten(),
    label=label
)


p = figure(title="Matrix plots of stabilities landscapes for ABX3 perovskites",
           x_axis_location="above", tools="hover,save",
           x_range=A, y_range=B,
           tooltips = [('names', '@label'), ('Delta H_f', '@count eV/atom')])

p.width = 800
p.height = 800
p.grid.grid_line_color = None
p.axis.axis_line_color = None
p.axis.major_tick_line_color = None
p.axis.major_label_text_font_size = "12px"
p.axis.major_label_standoff = 0
p.xaxis.major_label_orientation = np.pi/3

p.rect('xname', 'yname', 0.9, 0.9, source=data,
       line_color='None',alpha='alphas',#color=colors,
       hover_line_color='black')

show(p)

ERROR:bokeh.core.validation.check:E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name. This could either be due to a misspelling or typo, or due to an expected column being missing. : key "line_color" value "None" [renderer: GlyphRenderer(id='26561', ...)]
